In [ ]:
# VYTVORENIE DATASETU overcontact_one_curve.pkl

In [1]:
# BLOK 1
# Importovanie kniznic.

import numpy as np
import pandas as pd
import sqlite3
import io

In [3]:
# BLOK 2
# Funkcia pre vyber svetelnej krivky z nespracovanych dat. Zadefinovanie cesty k databaze (subor .db).

def get_curve(txt):
    out = io.BytesIO(txt)
    out.seek(0)
    return np.load(out)

FILEPATH = '../../../data-upjs/overcontact.db'

In [4]:
# BLOK 3 
# Vytvorenie pripojenia na subor. Vypis vsetkych tabuliek v databaze.

conn = sqlite3.connect(FILEPATH)

sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor = conn.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('parameters',), ('curves',), ('auxiliary',)]


In [5]:
# BLOK 4
# Nacitanie parametrov.

df_parameters = pd.read_sql_query("SELECT * FROM parameters", conn)

In [6]:
# BLOK 5
# Nacitanie svetelnych kriviek.

df_curves = pd.read_sql_query("SELECT * FROM curves", conn)

In [7]:
# BLOK 6
# Vytvorenie tabulky. Jeden riadok = jeden system zakrytovych premennych hviezd v 13 filtroch.

df1 = df_curves[["id", "Bessell_U", "Bessell_B", "Bessell_V", "Bessell_R", "Bessell_I", "SLOAN_u", "SLOAN_g", "SLOAN_r", "SLOAN_i", "SLOAN_z", "Kepler", "GaiaDR2", "TESS"]]
df2 = df_parameters[["id","primary__t_eff", "secondary__t_eff", "inclination", "mass_ratio", "primary__surface_potential", "secondary__surface_potential"]]
df_merged = pd.merge(df1,df2,on="id")
df_merged["Bessell_U"] = df_merged["Bessell_U"].apply(get_curve)
df_merged["Bessell_V"] = df_merged["Bessell_V"].apply(get_curve)
df_merged["Bessell_R"] = df_merged["Bessell_R"].apply(get_curve)
df_merged["Bessell_I"] = df_merged["Bessell_I"].apply(get_curve)
df_merged["Bessell_B"] = df_merged["Bessell_B"].apply(get_curve)
df_merged["SLOAN_u"] = df_merged["SLOAN_u"].apply(get_curve)
df_merged["SLOAN_g"] = df_merged["SLOAN_g"].apply(get_curve)
df_merged["SLOAN_r"] = df_merged["SLOAN_r"].apply(get_curve)
df_merged["SLOAN_i"] = df_merged["SLOAN_i"].apply(get_curve)
df_merged["SLOAN_z"] = df_merged["SLOAN_z"].apply(get_curve)
df_merged["Kepler"] = df_merged["Kepler"].apply(get_curve)
df_merged["GaiaDR2"] = df_merged["GaiaDR2"].apply(get_curve)
df_merged["TESS"] = df_merged["TESS"].apply(get_curve)

In [10]:
# BLOK 7
# Vytvorenie tabulky. Jeden riadok = jedna svetelna krivka podla jedneho filtra.

df_final = pd.melt(df_merged, 
                   id_vars=["id","primary__t_eff", "secondary__t_eff", "inclination", "mass_ratio", 
                            "primary__surface_potential", "secondary__surface_potential"],
                   var_name="filter", value_name="curve")

In [ ]:
# BLOK 8
# Uzavretie pripojenia na databazu. Ulozenie dat v podobe tabulky do suboru .pkl.

conn.close()
df_final.to_pickle("overcontact_one_curve.pkl") 